In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pandas.api.types import CategoricalDtype

In [2]:
men_matches = pd.read_csv('../data/all_matches_atp.csv')
men_matches = pd.DataFrame(men_matches)
men_matches.drop(men_matches.columns[0], axis=1, inplace=True)

women_matches = pd.read_csv('../data/all_matches_wta.csv')
women_matches = pd.DataFrame(women_matches)
women_matches.drop(women_matches.columns[0], axis=1, inplace=True)

/var/folders/d3/9chl4s4x2w1dxn5y2k9m88fc0000gn/T/ipykernel_9312/3778916847.py:5: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  women_matches = pd.read_csv('../data/all_matches_wta.csv')


In [3]:
atp_players = pd.read_csv('../data/tennis_atp/atp_players.csv')
atp_players['player_name'] = atp_players.apply(lambda row: str(row['name_first']) + " " + str(row['name_last']), axis=1)
wta_players = pd.read_csv('../data/tennis_wta/wta_players.csv')
wta_players['player_name'] = wta_players.apply(lambda row: str(row['name_first']) + " " + str(row['name_last']), axis=1)

/var/folders/d3/9chl4s4x2w1dxn5y2k9m88fc0000gn/T/ipykernel_9312/3930491513.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  atp_players = pd.read_csv('../data/tennis_atp/atp_players.csv')
/var/folders/d3/9chl4s4x2w1dxn5y2k9m88fc0000gn/T/ipykernel_9312/3930491513.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  wta_players = pd.read_csv('../data/tennis_wta/wta_players.csv')


In [4]:
def map_tourney_level_wta(tourney_level):
    if tourney_level == 'T1':
        return 'PM'
    elif tourney_level == 'T2':
        return 'P'
    else:
        return tourney_level

women_matches['tourney_level'] = women_matches['tourney_level'].map(map_tourney_level_wta)

In [5]:
def filter_matches(df):
    return df[df['tourney_level'].isin(['A', 'G', 'M', 'F', 'PM', 'P'])]
    

In [6]:
men_matches = filter_matches(men_matches)
women_matches = filter_matches(women_matches)

In [7]:
def get_decade(year):
    if str(year)[2] == '9':
        return '90s'
    elif str(year)[2] == '0':
        return '00s'
    elif str(year)[2] == '1':
        return '10s'
    elif str(year)[2] == '2':
        return '20s'
    else:
        return np.nan
    
def set_dates(df):
    df['tourney_date'] = pd.to_datetime(df['tourney_date'], format='%Y%m%d')
    df['tourney_year'] = df.tourney_date.dt.year
    df['decade'] = df['tourney_year'].apply(get_decade)
    return df 

men_matches = set_dates(men_matches)
women_matches = set_dates(women_matches)

In [13]:
def get_player_name(row):
    if row['winner_name'] is not np.nan:
        return row['winner_name']
    else:
        return row['loser_name']

def get_match_counts_player(df, conditions=None):
    if conditions is None:
        groupByList = []
    else:
        groupByList = conditions
    won = (df.groupby(['winner_id','winner_name'] + groupByList).size().reset_index(name='matches_won')
           .rename(columns={'winner_name': 'player_name', 'winner_id':'player_id'}))
    lost = (df.groupby(['loser_id','loser_name'] + groupByList).size().reset_index(name='matches_lost')
            .rename(columns={'loser_name': 'player_name', 'loser_id': 'player_id'}))
    total = (pd.merge(left=won, right=lost, how='outer', on=['player_id', 'player_name'] + groupByList)
                                   .fillna({'matches_won': 0, 'matches_lost': 0}))
    return total

In [163]:
def get_titles(df, tourney_levels):
    rename_dict = {x:x+'_titles' for x in tourney_levels}
    rename_dict['winner_name'] = 'player_name'
    rename_dict['winner_id'] = 'player_id'
    return df[df['round'] == 'F'].groupby(['winner_id','winner_name', 'tourney_level']).size().reset_index(name='count').pivot(columns='tourney_level', values='count', index=['winner_id', 'winner_name']).reset_index().fillna(0).rename(columns=rename_dict)

In [252]:
def get_career_years(df):
    return pd.concat(
        [
            df.groupby(['winner_id','winner_name']).agg(earliest=('tourney_year', 'min'), latest=('tourney_year', 'max')).reset_index().rename(columns={'winner_name': 'player_name', 'winner_id':'player_id'}),
            df.groupby(['loser_id','loser_name']).agg(earliest=('tourney_year', 'min'), latest=('tourney_year', 'max')).reset_index().rename(columns={'loser_name':'player_name', 'loser_id':'player_id'})
        ],
        axis=0 
    ).groupby(['player_id','player_name']).agg(career_start=('earliest', 'min'), career_end=('latest', 'max')).reset_index()

In [254]:
def get_player_profiles(match_df, player_df):
    match_counts = get_match_counts_player(match_df)
    match_counts['win_rate'] = match_counts['matches_won'] / (match_counts['matches_lost'] + match_counts['matches_won'])
    titles = get_titles(match_df, match_df.tourney_level.unique().tolist())
    career_years = get_career_years(match_df)
    player_data = player_df[['player_id', 'hand','dob','ioc','height']]
    return pd.merge(
        pd.merge(
            pd.merge(
                match_counts, 
                titles,
                on=['player_id', 'player_name'],
                how='left'
            ).fillna(0), 
            career_years,
            on=['player_id', 'player_name'],
        ),
        player_data,
        on='player_id'
    )

In [255]:
get_player_profiles(men_matches, atp_players).to_csv('../data/atp_player_profile.csv')
get_player_profiles(women_matches, wta_players).to_csv('../data/wta_player_profile.csv')

In [47]:
def get_avg_rank_player(match_df):
    player_matches = pd.concat(
                    [
                        match_df[['winner_id', 'winner_name', 'tourney_level', 'surface', 'decade', 'winner_rank']].rename(
                            columns={'winner_name': 'player_name', 'winner_rank': 'rank', 'winner_id':'player_id'}),
                         match_df[['loser_id', 'loser_name', 'tourney_level', 'surface', 'decade', 'loser_rank']].rename(
                             columns={'loser_name': 'player_name', 'loser_rank': 'rank', 'loser_id': 'player_id'})
                    ],
                    ignore_index=True
                )
    ranks_surf_tourney = player_matches.groupby(['player_id', 'player_name', 'tourney_level', 'surface', 'decade']).agg(avg_rank=('rank', 'mean')).reset_index()
    ranks_surf = player_matches.groupby(['player_id', 'player_name', 'surface', 'decade']).agg(avg_rank=('rank', 'mean')).reset_index()
    ranks_surf['tourney_level'] = 'All'
    ranks_tourney = player_matches.groupby(['player_id', 'player_name', 'tourney_level', 'decade']).agg(avg_rank=('rank', 'mean')).reset_index()
    ranks_tourney['surface'] = 'All'
    ranks_all = player_matches.groupby(['player_id', 'player_name', 'decade']).agg(avg_rank=('rank', 'mean')).reset_index()
    ranks_all['surface'] = 'All'
    ranks_all['tourney_level'] = 'All'
    
    return pd.concat(
        [
            ranks_surf_tourney[['player_id', 'player_name', 'decade', 'tourney_level', 'surface', 'avg_rank']],
            ranks_surf[['player_id', 'player_name', 'decade', 'tourney_level', 'surface', 'avg_rank']],
            ranks_tourney[['player_id', 'player_name', 'decade', 'tourney_level', 'surface', 'avg_rank']],
            ranks_all[['player_id', 'player_name', 'decade', 'tourney_level', 'surface', 'avg_rank']],
        ],
        axis = 0,
        ignore_index=True
    )

In [46]:
match_df = men_matches


,player_id,player_name,decade,tourney_level,surface,avg_rank
0,100282,Guillermo Vilas,90s,A,Clay,427.000000
1,100284,Jimmy Connors,90s,A,Carpet,443.200000
2,100284,Jimmy Connors,90s,A,Clay,362.777778
3,100284,Jimmy Connors,90s,A,Grass,NaN
4,100284,Jimmy Connors,90s,A,Hard,142.282051
...,...,...,...,...,...,...
30337,212021,Martin Landaluce,20s,All,All,497.333333
30338,212044,Yi Zhou,20s,All,All,599.428571
30339,212063,Matej Dodig,20s,All,All,310.000000
30340,212082,Joel Schwaerzler,20s,All,All,369.500000


In [12]:
def compute_score(row):
    num_played = row['matches_won'] + row['matches_lost']
    score = np.log(num_played + 1)
    score *= row['win_rate']
    score *= 1 / (np.log(row['avg_rank'] + np.e))
    return score


In [54]:
def get_filter_player_data(match_df, player_df):
    match_counts = get_match_counts_player(match_df, ['tourney_level', 'surface', 'decade'])
    match_counts_all = get_match_counts_player(match_df, ['decade'])
    match_counts_all['surface'] = 'All'
    match_counts_all['tourney_level'] = 'All'
    match_counts_all_surfaces = get_match_counts_player(match_df, ['decade', 'tourney_level'])
    match_counts_all_surfaces['surface'] = 'All'
    match_counts_all_tourneys = get_match_counts_player(match_df, ['decade', 'surface'])
    match_counts_all_tourneys['tourney_level'] = 'All'
    match_counts = pd.concat(
        [
            match_counts[['player_id', 'player_name', 'tourney_level', 'surface','decade', 'matches_won', 'matches_lost']],
            match_counts_all[['player_id', 'player_name', 'tourney_level', 'surface','decade', 'matches_won', 'matches_lost']], 
            match_counts_all_tourneys[['player_id', 'player_name', 'tourney_level', 'surface','decade', 'matches_won', 'matches_lost']],
            match_counts_all_surfaces[['player_id', 'player_name', 'tourney_level', 'surface','decade', 'matches_won', 'matches_lost']]
        ],
        axis=0,
        ignore_index=True
    )
    match_counts['win_rate'] = match_counts['matches_won'] / (match_counts['matches_lost'] + match_counts['matches_won'])
    player_data = pd.merge(match_counts, player_df[['player_id', 'player_name','hand','ioc']], on=['player_id', 'player_name'])
    avg_rank = get_avg_rank_player(match_df)
    player_data = pd.merge(player_data, avg_rank, on=['player_id','player_name', 'tourney_level', 'surface', 'decade'], how='left')
    player_data['score'] = player_data.apply(compute_score, axis=1)
    player_data.dropna(subset=['score'], inplace=True)
    return player_data

In [59]:
get_filter_player_data(men_matches,atp_players).to_csv('../data/atp_filter_player_data.csv')
get_filter_player_data(women_matches,wta_players).to_csv('../data/wta_filter_player_data.csv')

In [191]:
def surface_win_rates(match_df):
    match_counts = get_match_counts_player(match_df, ['surface'])
    match_counts['win_rate'] = match_counts.apply(lambda row: row['matches_won'] / (row['matches_won'] + row['matches_lost']) * 100, axis=1)
    return match_counts

In [220]:
surface_win_rates(men_matches).to_csv('../data/atp_win_rates.csv')
surface_win_rates(women_matches).to_csv('../data/wta_win_rates.csv')

In [8]:
def get_rivalries(match_df):
    player_opponents = pd.concat(
        [
            match_df[['winner_id','winner_name', 'surface','winner_rank', 'loser_name','loser_id']]
            .rename(columns={'winner_name': 'player_name', 'winner_id':'player_id', 'winner_rank': 'rank', 'loser_name': 'opponent', 'loser_id': 'opponent_id'}),
    
            match_df[['loser_name','loser_id', 'surface', 'loser_rank', 'winner_name', 'winner_id']]
            .rename(columns={'loser_name': 'player_name', 'loser_id':'player_id', 'loser_rank': 'rank', 'winner_name': 'opponent', 'winner_id':'opponent_id'})
        ],
        ignore_index=True
    )

    match_counts = (
        player_opponents.groupby(['player_id','player_name', 'opponent', 'opponent_id','surface'])
        .size()
        .reset_index(name='match_count')
    )
    
    top_opponents = (
        match_counts.sort_values(['player_id','match_count'], ascending=[True, False])
            .groupby(['player_id'])
        .head(10)
    )
    return top_opponents

In [9]:
get_rivalries(men_matches).to_csv('../data/atp_rivalries2.csv')
get_rivalries(women_matches).to_csv('../data/wta_rivalries2.csv')

In [217]:
get_rivalries(men_matches).to_csv('../data/atp_rivalries.csv')
get_rivalries(women_matches).to_csv('../data/wta_rivalries.csv')

In [8]:
def player_performance(df):
    round_order = ['RR', 'BR', 'ER', 'R128', 'R64', 'R32', 'R16', 'QF', 'SF', 'F']
    round_dtype = CategoricalDtype(categories=round_order, ordered=True)

    df['round'] = df['round'].astype(round_dtype)

    rounds = df.groupby(['winner_id','winner_name', 'tourney_year', 'tourney_level', 'tourney_name'])[
        'round'].max().reset_index()
    rounds = rounds.groupby(['winner_id','winner_name', 'tourney_year', 'tourney_level', 'round'],
                            observed=True).size().reset_index(name='match_count')
    max_rank_points = df.groupby(['winner_id','winner_name', 'tourney_year']).agg(
        best_rank=('winner_rank', 'min')).reset_index()
    player_perf = pd.merge(rounds, max_rank_points, on=['winner_id','winner_name', 'tourney_year'])
    player_perf['titles'] = player_perf.apply(lambda x: 0 if x['round'] != 'F' else x['match_count'], axis=1)
    player_perf.rename(columns={'winner_id': 'player_id', 'winner_name':'player_name'}, inplace=True)
    return player_perf

In [239]:
player_performance(women_matches).to_csv('../data/wta_player_perf.csv')
player_performance(men_matches).to_csv('../data/atp_player_perf.csv')

In [240]:
def match_breakdown_surface_tourney(df, threshold=3):
    count_sunburst = df.groupby(
        ['winner_id', 'winner_name', 'surface', 'tourney_level', 'tourney_name']).size().reset_index(name='count')
    count_sunburst.loc[count_sunburst['count'] <=threshold, 'tourney_name'] = 'Other'
    return count_sunburst.groupby(['winner_id','winner_name', 'surface', 'tourney_level', 'tourney_name']).sum('count').reset_index().rename(columns={'winner_id': 'player_id', 'winner_name':'player_name'})

In [243]:
match_breakdown_surface_tourney(men_matches).to_csv('../data/atp_prop_surface.csv')
match_breakdown_surface_tourney(women_matches).to_csv('../data/wta_prop_surface.csv')

In [244]:
def compute_player_stats(row):
    row['ace_pct'] = row['w_ace'] / row['w_svpt'] * 100 if row['w_svpt'] > 0 else np.nan
    row['1stIn_pct'] = row['w_1stIn'] / row['w_svpt'] * 100 if row['w_svpt'] > 0 else np.nan
    row['1stWon_pct'] = row['w_1stWon'] / row['w_1stIn'] * 100 if row['w_1stIn'] > 0 else np.nan
    row['2ndWon_pct'] = row['w_2ndWon'] / (row['w_svpt'] - row['w_1stIn'] - row['w_df']) * 100 if (row['w_svpt'] - row[
        'w_1stIn'] - row['w_df']) > 0 else np.nan
    row['df_pct'] = row['w_df'] / row['w_svpt'] * 100 if row['w_svpt'] > 0 else np.nan
    row['bpSaved_pct'] = row['w_bpSaved'] / row['w_bpFaced'] * 100 if row['w_bpFaced'] > 0 else np.nan
    return row

def compute_all_stats(df, groupSurface=True):
    df = df.apply(compute_player_stats, axis=1)
    if groupSurface:
        statsDf= df.groupby(['winner_id','winner_name', 'surface'])[["ace_pct", '1stIn_pct', '1stWon_pct', '2ndWon_pct', 'df_pct', 'bpSaved_pct']].mean().reset_index()
    else: 
        statsDf= df.groupby(['winner_id','winner_name'])[["ace_pct", '1stIn_pct', '1stWon_pct', '2ndWon_pct', 'df_pct', 'bpSaved_pct']].mean().reset_index()
    statsDf.rename(columns={'winner_id': 'player_id', 'winner_name':'player_name'}, inplace=True)
    return statsDf

In [246]:
menstats = compute_all_stats(men_matches)
womenstats = compute_all_stats(women_matches)

In [249]:
menstats.to_csv('../data/atp_player_stats.csv')
womenstats.to_csv('../data/wta_player_stats.csv')